# Challenge 2

In this challenge we will continue working with `Pokemon.csv`. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemons are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.

## Provlem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# import libraries
import numpy as np
import pandas as pd

In [2]:
# importing the dataset and inspect the first few rows of the data
pokemon = pd.read_csv('Pokemon.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemons?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
# your code here

pokemon['Total_2']=pokemon['HP']+pokemon['Attack']+pokemon['Defense']+pokemon['Sp. Atk']+pokemon['Sp. Def']+pokemon['Speed']

pokemon['Total_2'].equals(pokemon['Total'])

#Yes

True

## Provlem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](one-hot-encoding.png)

In [7]:
# your code here
# pokemon_names=['Grass','Poison','Fire','Flying','Dragon','Water','Bug','Normal','Electric','Ground','Fairy','Fighting','Psychic','Rock','Steel','Ice','Ghost','Dark'
# ]
# df_Type=pd.get_dummies(pokemon_names,prefix='T')
# pd.concat([df[['Type 1']],df_Type],axis=1).head
pokemon['Type 2'] = np.where(pokemon['Type 2'].isnull(), 'NaN', pokemon['Type 2'])
pokemon['Type 1'] = pd.Categorical(pokemon['Type 1'])
dfDummies = pd.get_dummies(pokemon['Type 1'])
df1 = pd.concat([pokemon, dfDummies], axis=1)
df1=df1[['Grass','Poison','Fire','Flying','Dragon','Water','Bug','Normal','Electric','Ground','Fairy','Fighting','Psychic','Rock','Steel','Ice','Ghost','Dark']]
pokemon['Type 2'] = pd.Categorical(pokemon['Type 2'])
dfDummies = pd.get_dummies(pokemon['Type 2'])
df2 = pd.concat([pokemon, dfDummies], axis=1)
df2=df2.drop(['NaN'],axis=1)
df2=df2[['Grass','Poison','Fire','Flying','Dragon','Water','Bug','Normal','Electric','Ground','Fairy','Fighting','Psychic','Rock','Steel','Ice','Ghost','Dark']]
df2


# df1.reindex_like(df2).fillna(0) + df2.fillna(0).fillna(0)

# df1 = pd.DataFrame({'val':{'Grass':1,'Poison':2,'Fire':3,'Flying':4,'Dragon':5,'Water':6,'Bug':7,'Normal':8,'Electric':9,'Ground':10,'Fairy':11,'Fighting':12,'Psychic':13,'Rock':14,'Steel':15,'Ice':16,'Ghost':17,'Dark':18}})
# df2 = pd.DataFrame({'val':{'Grass':1,'Poison':2,'Fire':3,'Flying':4,'Dragon':5,'Water':6,'Bug':7,'Normal':8,'Electric':9,'Ground':10,'Fairy':11,'Fighting':12,'Psychic':13,'Rock':14,'Steel':15,'Ice':16,'Ghost':17,'Dark':18}})

df1.add(df2, fill_value=0)

d=df1+df2
d

,Grass,Poison,Fire,Flying,Dragon,Water,Bug,Normal,Electric,Ground,Fairy,Fighting,Psychic,Rock,Steel,Ice,Ghost,Dark
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
796,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
797,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
798,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# pokemon['Type 2'] = np.where(pokemon['Type 2'].isnull(), 'NaN', pokemon['Type 2'])


# enc = OneHotEncoder(handle_unknown='ignore')
# X=pokemon[['Type 1','Type 2']]

# enc.fit(X)

# enc.categories_
# enc.transform([['Type 1', 1], ['Type 2', 1]]).toarray()

In [ ]:
# from sklearn.compose import ColumnTransformer
# pokemon['Type 2'] = np.where(pokemon['Type 2'].isnull(), 'NaN', pokemon['Type 2'])
# X=pokemon[['Type 1','Type 2']]
#                                        # Leave the rest of the columns untouched


# ohe = OneHotEncoder(categories='T')
# feature_arr = ohe.fit_transform(X).toarray()
# feature_arr
# feature_labels = ohe.categories_
# feature_labels = np.array(feature_labels).ravel()
# features = pd.DataFrame(feature_arr, columns=feature_labels)
# features

In [ ]:
# pokemon['Type 2'] = np.where(pokemon['Type 2'].isnull(), 'NaN', pokemon['Type 2'])
# X=pokemon[['Type 1','Type 2']]

# train = pd([X.get(['Type 1','Type 2']), pd.get_dummies(X['Type 1'], prefix='T1')+ pd.get_dummies(X['Type 2'], prefix='T2')],axis=1)

# train



## Provlem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [8]:
# your code here
dc= pd.concat([pokemon['Total'], d],axis=1)
dc
correlaciones=dc.corr()
correlaciones



,Total,Grass,Poison,Fire,Flying,Dragon,Water,Bug,Normal,Electric,Ground,Fairy,Fighting,Psychic,Rock,Steel,Ice,Ghost,Dark
Total,1.000000,-0.052592,-0.090441,0.078726,0.059383,0.229705,-0.021665,-0.145781,-0.105331,0.020971,0.015060,-0.036698,0.077786,0.124688,0.032731,0.109703,0.060248,0.003641,0.056154
Grass,-0.052592,1.000000,0.110372,-0.108248,-0.081366,-0.078818,-0.126894,-0.034430,-0.117156,-0.078818,-0.097033,-0.048756,-0.051171,-0.094009,-0.072829,-0.061537,-0.027476,0.075316,-0.048339
Poison,-0.090441,0.110372,1.000000,-0.085471,-0.067950,-0.055525,-0.073986,0.121210,-0.110800,-0.074838,-0.053878,-0.066495,-0.039613,-0.103195,-0.081036,-0.074036,-0.064726,0.008736,-0.018227
Fire,0.078726,-0.108248,-0.085471,1.000000,-0.028855,-0.038069,-0.114850,-0.060536,-0.085097,-0.057104,-0.022621,-0.067651,0.051130,-0.061243,-0.064677,-0.056108,-0.065851,-0.013459,-0.020368
Flying,0.059383,-0.081366,-0.067950,-0.028855,1.000000,0.026237,-0.092030,0.064571,0.125506,-0.020407,-0.060578,-0.052669,-0.086119,-0.063872,-0.048221,-0.081400,-0.049498,-0.061555,-0.022164
Dragon,0.229705,-0.078818,-0.055525,-0.038069,0.026237,1.000000,-0.083285,-0.081200,-0.098702,-0.024000,0.052430,-0.035541,-0.068775,-0.026557,-0.032360,-0.044417,0.015173,-0.019410,-0.003963
Water,-0.021665,-0.126894,-0.073986,-0.114850,-0.092030,-0.083285,1.000000,-0.123983,-0.154994,-0.069109,-0.006844,-0.067703,-0.073778,-0.099639,0.011447,-0.096131,0.000242,-0.077314,-0.028548
Bug,-0.145781,-0.034430,0.121210,-0.060536,0.064571,-0.081200,-0.123983,1.000000,-0.120219,-0.045111,-0.063544,-0.072148,-0.048645,-0.111968,-0.003706,0.047178,-0.070229,-0.058915,-0.082062
Normal,-0.105331,-0.117156,-0.110800,-0.085097,0.125506,-0.098702,-0.154994,-0.120219,1.000000,-0.067737,-0.102044,-0.001720,-0.071687,-0.112381,-0.106877,-0.097645,-0.085366,-0.094420,-0.099751
Electric,0.020971,-0.078818,-0.074838,-0.057104,-0.020407,-0.024000,-0.069109,-0.045111,-0.067737,1.000000,-0.059420,-0.035541,-0.068775,-0.091928,-0.072188,-0.001346,-0.033382,-0.041592,-0.067375


In [9]:
s = correlaciones.unstack()
so = s.sort_values(ascending=False)
so[18:]

Total   Total     1.000000
        Dragon    0.229705
Dragon  Total     0.229705
Normal  Flying    0.125506
Flying  Normal    0.125506
                    ...   
Water   Grass    -0.126894
Total   Bug      -0.145781
Bug     Total    -0.145781
Water   Normal   -0.154994
Normal  Water    -0.154994
Length: 343, dtype: float64

In [10]:
pokemon[(pokemon['Total']==780)]


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total_2
163,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True,780
164,150,MewtwoMega Mewtwo Y,Psychic,NaN,780,106,150,70,194,120,140,1,True,780
426,384,RayquazaMega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True,780


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here